In [ ]:
%run "0a_Workspace_setup.ipynb"

## 1. Read NHM Subbasin HRU and Segment geodatabases.
This reads three layers (<b>nhru, amd nsegments</b>) into GeoPandas as DataFrames (_df) and if geometry is included (_gdb).
<b>Note:</b> Layer npoigages includes the poi gages that were included in the model and are limited. Since pois will be added to the model paramter file, we provide another method of for retrieving poi metadata, such as latitude (lat) and longitude (lon), for pois listed in the parameter file that uses NWIS and a supplimental gage ref table for gages that do not occur in NWIS. Locations may NOT be located exactly on the NHM segment. The POIs' assigned segment is displayed in the popup window when the gage icon is clicked.

#### Coordinate system and projection housekeeping
Note: Projections are inherited with geometry from the HRUs geodatabase. The NHM uses the **NAD 1983 USGS Contiguous USA Albers** projection EPSG# 102039. The geometry units of this projection are not useful for many notebook packages. The **geodatabases are  reprojected to World Geodetic System 1984**.<br>
<br>
Options:
- crs = 3857, WGS 84 / Pseudo-Mercator - Spherical Mercator, Google Maps, OpenStreetMap, Bing, ArcGIS, ESRI.<br> is also an option
- crs = 4326, WGS 84 - WGS84 - World Geodetic System 1984, used in GPS

In [ ]:
crs = 4326  # define the new projection that will give lat/lon in units needed for folium mapping
# print(poi_gdb.crs) #check the projection of the file

In [ ]:
# Read in the HRU geodatabase
if GIS_format == ".gpkg":
    hru_gdb = gpd.read_file(
        f"{model_dir}/GIS/model_layers.gpkg", layer="nhru"
    )  # Reads HRU file to Geopandas.

if GIS_format == ".shp":
    hru_gdb = gpd.read_file(
        f"{model_dir}/GIS/model_nhru.shp"
    )  # Reads HRU file to Geopandas.
    hru_gdb = hru_gdb.set_index("nhm_id", drop=False).fillna(
        0
    )  # Set an index for HRU geodatabase.
    hru_gdb.index.name = "index"  # Index column must be renamed of the hru

hru_gdb = hru_gdb.to_crs(crs)  # reprojects to the defined crs projection

# hru_gdb.head(10) #prints the first 10
# hru_gdb.sample(10) #prints randam 10

In [ ]:
# Read in the Segment geodatabase
if GIS_format == ".gpkg":
    seg_gdb = gpd.read_file(
        f"{model_dir}/GIS/model_layers.gpkg", layer="nsegment"
    ).fillna(
        0
    )  # Reads segemnt file to Geopandas.

if GIS_format == ".shp":
    seg_gdb = gpd.read_file(f"{model_dir}/GIS/model_nsegment.shp").fillna(0)
    seg_gdb = seg_gdb.set_index(
        "nhm_seg", drop=False
    )  # Set an index for segment geodatabase(GIS)
    seg_gdb.index.name = "index"  # Index column must be renamed of the hru

seg_gdb = seg_gdb.to_crs(crs)  # reprojects to the defined crs projection

# seg_gdb.head(3)

## 2. Merge HRU parameters from the parameter file with HRU GeoDataFrame
Extracts HRU latitude and longitude from the params file using pyPRMS and creates a dataframe. This is useful as a cross-check for HRU location and in later notebook leaflet applications.

In [ ]:
# List of parameters values to retrieve for the HRUs.
hru_params = [
    "hru_lat",  # the latitude if the hru centroid
    "hru_lon",  # the longitude if the hru centroid
    "hru_area",
    "hru_segment_nhm",  # The nhm_id of the segment recieving flow from the HRU
]  # List any param here that is dimensioned by nHRU. Not intended for by nmonth X nHRU

In [ ]:
# Create list of calibration parameters
nhru_params = [
    "carea_max",
    "emis_noppt",
    "fastcoef_lin",
    "freeh2o_cap",
    "gwflow_coef",
    "potet_sublim",
    "rad_trncf",
    "slowcoef_sq",
    "smidx_coef",
    "smidx_exp",
    "snowinfil_max",
    "soil2gw_max",
    "soil_moist_max",
    "soil_rechr_max_frac",
    "ssr2gw_exp",
    "ssr2gw_rate",
]  # List any output param here.

nhru_nmonths_params = [
    "adjmix_rain",
    "cecn_coef",
    "jh_coef",
    "tmax_allrain_offset",
    "tmax_allsnow",
    "radmax",
    "rain_cbh_adj",
    "snow_cbh_adj",
    "tmax_cbh_adj",
    "tmin_cbh_adj",
]  # List any output param here.
cal_hru_params = nhru_params + nhru_nmonths_params
gdb_hru_params = hru_params + nhru_params + nhru_nmonths_params

In [ ]:
# Create a dataframe for parameter values
first = True
for vv in gdb_hru_params:
    if (
        first
    ):  # this creates the first iteration for the following iterations to concantonate to
        df = pdb.get_dataframe(vv)
        first = False
    else:
        df = pd.concat([df, pdb.get_dataframe(vv)], axis=1)  # , ignore_index=True)

df.reset_index(inplace=True)
df["model_idx"] = (
    df.index + 1
)  #'model_idx' created here is the order of the parameters in the parameter file.
# df

In [ ]:
# Join the HRU params values to the HRU geodatabase using Merge
hru_gdb = pd.merge(df, hru_gdb, on="nhm_id")
# hru_gdb.round(8)

In [ ]:
# Create a Goepandas GeoDataFrame for the HRU geodatabase
hru_gdf = gpd.GeoDataFrame(
    hru_gdb, geometry="geometry"
)  # Creates a Geopandas GeoDataFrame

## 3. Merge segment parameter values from the parameter file with segment GeoDataFrame

In [ ]:
# List of parameters values to retrieve for the segments.
seg_params = ["tosegment_nhm", "tosegment", "seg_length", "obsin_segment"]

In [ ]:
# Create a dataframe for parameter values
first = True
for vv in seg_params:
    if first:
        df = pdb.get_dataframe(vv)
        first = False
    else:
        df = pd.concat([df, pdb.get_dataframe(vv)], axis=1)  # , ignore_index=True)

df.reset_index(inplace=True)
df["model_idx"] = df.index + 1
df.index.name = "index"  # Index column must be renamed

In [ ]:
# Join the HRU params values to the HRU geodatabase using Merge
seg_gdb = pd.merge(df, seg_gdb, on="nhm_seg")

In [ ]:
# Create a Goepandas GeoDataFrame for the HRU geodatabase
seg_gdf = gpd.GeoDataFrame(seg_gdb, geometry="geometry")
seg_gdf.head()

## Create POI DataFrame (poi_df) for gages included in the NHMx parameter file (poi_gages).

#### Get POI-related parameters and reshape as a dataframe

In [ ]:
poi = pdb["poi_gage_id"].as_dataframe
poi = poi.merge(pdb["poi_gage_segment"].as_dataframe, left_index=True, right_index=True)
poi = poi.merge(pdb["poi_type"].as_dataframe, left_index=True, right_index=True)
poi = poi.merge(
    pdb["nhm_seg"].as_dataframe, left_on="poi_gage_segment", right_index=True
)
# poi.info()

#### Get poi metadata from NWIS.


In [ ]:
# imports
import datetime
import re
from io import StringIO
import sys
from collections import OrderedDict
from urllib.request import urlopen, Request
from urllib.error import HTTPError

In [ ]:
# start and end dates are for the date span for gage activity.
st_date = datetime.datetime(1930, 1, 1)
en_date = datetime.datetime(2022, 12, 31)

In [ ]:
# NWIS data pull functions from pyPRMS
base_url = "http://waterservices.usgs.gov/nwis"

t1 = re.compile("^#.*$\n?", re.MULTILINE)  # remove comment lines
t2 = re.compile("^5s.*$\n?", re.MULTILINE)  # remove field length lines


def get_nwis_site_fields():
    """Get NWIS streamgage field information.

    :returns: dictionary of field_name, datatype pairs
    """
    # Retrieve a single station and pull out the field names and data types
    stn_url = (
        f"{base_url}/site/?format=rdb&sites=01646500&siteOutput=expanded&"
        + "siteStatus=active&parameterCd=00060&siteType=ST"
    )

    response = urlopen(stn_url)
    encoding = response.info().get_param("charset", failobj="utf8")
    streamgage_site_page = response.read().decode(encoding)

    # Strip the comment lines and field length lines from the result
    streamgage_site_page = t1.sub("", streamgage_site_page, 0)

    # nwis_dtypes = t2.findall(streamgage_site_page)[0].strip('\n').split('\t')
    nwis_fields = StringIO(streamgage_site_page).getvalue().split("\n")[0].split("\t")

    nwis_final = {}
    for fld in nwis_fields:
        code = fld[-2:]
        if code in ["cd", "no", "nm", "dt"]:
            nwis_final[fld] = np.str_
        elif code in ["va"]:
            nwis_final[fld] = np.float32
        else:
            nwis_final[fld] = np.str_

    return nwis_final


def _retrieve_from_nwis(url):
    """Get streamgage site page for given URL.

    :param url: URL to streamgage page on NWIS
    :returns: streamgage site page
    """
    response = urlopen(url)
    encoding = response.info().get_param("charset", failobj="utf8")
    streamgage_site_page = response.read().decode(encoding)

    # Strip the comment lines and field length lines from the result
    streamgage_site_page = t1.sub("", streamgage_site_page, 0)
    streamgage_site_page = t2.sub("", streamgage_site_page, 0)

    return streamgage_site_page


def get_nwis_sites(stdate, endate, sites=None, regions=None):
    """Get NWIS streamgage site information

    :param stdate: start date for extraction
    :param endate: end date for extraction
    :param sites: streamgage(s) to pull from NWIS
    :param regions: region(s) to pull from NWIS
    :returns: dataframe of streamgage site information
    """
    cols = get_nwis_site_fields()

    # Columns to include in the final dataframe
    include_cols = [
        "agency_cd",
        "site_no",
        "station_nm",
        "dec_lat_va",
        "dec_long_va",
        "dec_coord_datum_cd",
        "alt_va",
        "alt_datum_cd",
        "huc_cd",
        "drain_area_va",
        "contrib_drain_area_va",
    ]

    # Start with an empty dataframe
    nwis_sites = pd.DataFrame(columns=include_cols)

    url_pieces = OrderedDict()
    url_pieces["format"] = "rdb"
    url_pieces["startDT"] = stdate.strftime("%Y-%m-%d")
    url_pieces["endDT"] = endate.strftime("%Y-%m-%d")
    # url_pieces['huc'] = None
    url_pieces["siteOutput"] = "expanded"
    url_pieces["siteStatus"] = "all"
    url_pieces["parameterCd"] = "00060"  # Discharge
    url_pieces["siteType"] = "ST"
    # url_pieces['hasDataTypeCd'] = 'dv'

    # NOTE: If both sites and regions parameters are specified the sites
    #       parameter takes precedence.
    if sites is None:
        # No sites specified so default to HUC02-based retrieval
        url_pieces["huc"] = None

        if regions is None:
            # Default to HUC02 regions 1 thru 18
            regions = list(range(1, 19))
        if isinstance(regions, (list, tuple)):
            pass
        else:
            # Single region
            regions = [regions]
    else:
        # One or more sites were specified
        url_pieces["sites"] = None

        if isinstance(sites, (list, tuple)):
            pass
        else:
            # Single string, convert to list of sites
            sites = [sites]

    if "huc" in url_pieces:
        # for region in range(19):
        for region in regions:
            sys.stdout.write(f"\r  Region: {region:02}")
            sys.stdout.flush()

            url_pieces["huc"] = f"{region:02}"
            url_final = "&".join([f"{kk}={vv}" for kk, vv in url_pieces.items()])
            stn_url = f"{base_url}/site/?{url_final}"

            streamgage_site_page = _retrieve_from_nwis(stn_url)

            # Read the rdb file into a dataframe
            df = pd.read_csv(
                StringIO(streamgage_site_page),
                sep="\t",
                dtype=cols,
                usecols=include_cols,
            )

            # pandas append deprecated since v1.4
            # nwis_sites = nwis_sites.append(df, ignore_index=True)
            nwis_sites = pd.concat([nwis_sites, df], ignore_index=True)
            sys.stdout.write("\r                      \r")
    else:
        for site in sites:
            sys.stdout.write(f"\r  Site: {site} ")
            sys.stdout.flush()

            url_pieces["sites"] = site
            url_final = "&".join([f"{kk}={vv}" for kk, vv in url_pieces.items()])

            stn_url = f"{base_url}/site/?{url_final}"

            try:
                streamgage_site_page = _retrieve_from_nwis(stn_url)

                # Read the rdb file into a dataframe
                df = pd.read_csv(
                    StringIO(streamgage_site_page),
                    sep="\t",
                    dtype=cols,
                    usecols=include_cols,
                )

                # pandas append deprecated since v1.4
                # nwis_sites = nwis_sites.append(df, ignore_index=True)
                nwis_sites = pd.concat([nwis_sites, df], ignore_index=True)
            except HTTPError as err:
                if err.code == 404:
                    sys.stdout.write(
                        f"HTTPError: {err.code}, site does not meet criteria - SKIPPED\n"
                    )
            sys.stdout.write("\r                      \r")

    field_map = {
        "agency_cd": "poi_agency",
        "site_no": "poi_id",
        "station_nm": "poi_name",
        "dec_lat_va": "latitude",
        "dec_long_va": "longitude",
        "alt_va": "elevation",
        "drain_area_va": "drainage_area",
        "contrib_drain_area_va": "drainage_area_contrib",
    }

    nwis_sites.rename(columns=field_map, inplace=True)
    nwis_sites.set_index("poi_id", inplace=True)
    nwis_sites = nwis_sites.sort_index()

    return nwis_sites

In [ ]:
# Reads/Creates NWIS stations file if not already created
if nwis_gages_file.exists():
    col_names = [
        "poi_id",
        "poi_agency",
        "poi_name",
        "latitude",
        "longitude",
        #'dec_coord_datum_cd',
        #'elevation',
        #'alt_datum_cd',
        #'huc_cd',
        "drainage_area",
        "drainage_area_contrib",
    ]
    col_types = [
        np.str_,
        np.str_,
        np.str_,
        float,
        float,
        # np.str_,
        # float,
        # np.str_,
        # np.str_,
        float,
        float,
    ]
    cols = dict(
        zip(col_names, col_types)
    )  # Creates a dictionary of column header and datatype called below.

    nwis_gages_aoi = pd.read_csv(
        nwis_gages_file,
        dtype=cols,
        usecols=[
            "poi_id",
            "poi_agency",
            "poi_name",
            "latitude",
            "longitude",
            "drainage_area",
            "drainage_area_contrib",
        ],
    )  # sep='\t',
else:
    nwis_sites = get_nwis_sites(
        stdate=st_date, endate=en_date, regions=model_domain_regions
    )  # Pulls all sites in conus
    # Pull a list of sites
    # df1 = get_nwis_sites(stdate=st_date, endate=en_date, sites=['01012500', '01010000'])
    # Pull a list of regions
    # df1 = get_nwis_sites(stdate=st_date, endate=en_date, regions=['1', '2', '3'])

    # Make sites a geodatabase
    nwis_sites_gdf = gpd.GeoDataFrame(
        nwis_sites,
        geometry=gpd.points_from_xy(nwis_sites.longitude, nwis_sites.latitude),
        crs=crs,
    )

    # clip to the model domain
    nwis_gages_aoi = nwis_sites_gdf.clip(hru_gdf)
    nwis_gages_aoi.reset_index(inplace=True)
    nwis_gages_aoi.drop(columns=["geometry"], inplace=True)

    # trim off things we don't need
    nwis_gages_aoi.drop(
        columns=[
            "dec_coord_datum_cd",
            "elevation",
            "alt_datum_cd",
            "huc_cd",
            #'drainage_area',
            #'drainage_area_contrib'
        ],
        inplace=True,
    )

    # write out the file for later
    nwis_gages_aoi.to_csv(nwis_gages_file, index=False)  # , sep='\t')

In [ ]:
# nwis_sites_aoi.info()

In [ ]:
# Merge the temp dataframe (NHM poi information from NWIS) with the pois data from the parameter file.
poi = poi.merge(nwis_gages_aoi, left_on="poi_gage_id", right_on="poi_id", how="left")
poi_df = pd.DataFrame(poi)  # Creates a Pandas DataFrame

In [ ]:
# poi_df

## Create default_gages.csv for your model extraction.
##### This will be composed of the gages that are in the model, but also include NWIS gages that may be added later to the model. Time-series data for streamflow observations will be collected using this gage list.

#### Make a dataframe of the non-NWIS gages (if present) in the parameter file (poi_gages)

In [ ]:
# create a dataframe of the gages in the parameter file that are not USGS gages in NWIS
if pd.isnull(poi_df["poi_agency"]).values.any():
    non_NWIS_gages_from_poi_df = poi_df.loc[pd.isnull(poi_df["poi_agency"])]
    non_NWIS_gages_from_poi_df.drop(
        columns=["poi_id", "nhm_seg", "poi_gage_segment", "poi_type"], inplace=True
    )
    non_NWIS_gages_from_poi_df.rename(columns={"poi_gage_id": "poi_id"}, inplace=True)
    # non_NWIS_gages_from_poi_df

#### Make a dataframe for all non-NWIS gages (if present) in the parameter file and all NWIS gages in the model domain
##### Note: the NWIS gages in the poi_df (gages in the parameter file) should be in NWIS_sites_aoi df.

In [ ]:
sta_file_col_order = [
    "poi_id",
    "poi_agency",
    "poi_name",
    "latitude",
    "longitude",
    "drainage_area",
    "drainage_area_contrib",
    #'nhm_seg', 'poi_gage_segment', 'poi_type'
]
if pd.isnull(poi_df["poi_agency"]).values.any():
    temp = pd.concat([nwis_gages_aoi, non_NWIS_gages_from_poi_df], ignore_index=True)
    temp2 = temp[sta_file_col_order]

else:
    temp = nwis_gages_aoi.copy()
    temp2 = temp[sta_file_col_order]

#### Save the default station file (gage file) as a .csv file

In [ ]:
temp2.to_csv(default_gages_file, index=False)
# temp2.info()

In [ ]:
temp2

## Create gages.csv file using default_gages.csv file.
##### If there are gages in the parameter file that are not in NWIS (USGS gages), then latitude, longitude, and poi_name must be provided from another source, and appended to the "default_gages.csv" file. Once editing is complete, that file can be renamed "gages.csv"and will be used as the gages file. If NO gages.csv is made, the default_gages.csv will be used.

## Update gages_df if gages.csv was created

In [ ]:
# Read in station file columns needed (You may need to tailor this to the particular file.
col_names = [
    "poi_id",
    "poi_agency",
    "poi_name",
    "latitude",
    "longitude",
    "drainage_area",
    "drainage_area_contrib",
]
col_types = [np.str_, np.str_, np.str_, float, float, float, float]
cols = dict(
    zip(col_names, col_types)
)  # Creates a dictionary of column header and datatype called below.

if gages_file.exists():

    nwis_gages_aoi = pd.read_csv(nwis_gages_file, dtype=cols)
    gages_df = pd.read_csv(gages_file)

    # Make poi_id the index
    gages_df.set_index("poi_id", inplace=True)

    con.print(f"The appended gage file will be used and has {len(gages_df)} gages.")
else:
    gages_df = pd.read_csv(default_gages_file, dtype=cols)

    # Make poi_id the index
    gages_df.set_index("poi_id", inplace=True)

    con.print(f"The default gage file will be used and has {len(gages_df)} gages.")

## Update the poi_df metadata
#### If an improved/edited gages.csv file exists, then the poi_df metadata should also be updated.

In [ ]:
# Updates the non-usgs gages in the poi dataframe with metadata from the stations file (that was added or edited)
if gages_file.exists():
    for idx, row in poi_df.iterrows():
        if pd.isnull(row["poi_id"]):
            new_poi_id = row["poi_gage_id"]
            new_lat = gages_df.loc[
                gages_df.index == row["poi_gage_id"], "latitude"
            ].values[0]
            new_lon = gages_df.loc[
                gages_df.index == row["poi_gage_id"], "longitude"
            ].values[0]
            new_poi_agency = gages_df.loc[
                gages_df.index == row["poi_gage_id"], "poi_agency"
            ].values[0]
            new_poi_name = gages_df.loc[
                gages_df.index == row["poi_gage_id"], "poi_name"
            ].values[0]

            poi_df.loc[idx, "latitude"] = new_lat
            poi_df.loc[idx, "longitude"] = new_lon
            poi_df.loc[idx, "poi_id"] = new_poi_id
            poi_df.loc[idx, "poi_agency"] = new_poi_agency
            poi_df.loc[idx, "poi_name"] = new_poi_name

else:
    pass

##### All pois in the poi_df with missing data for lat, lon, and poi_name will be dropped from the poi_df

In [ ]:
# Print warning and drop poi's with missing data for lat, lon, and poi_name
missing_meta_df = poi_df[
    poi_df[["latitude", "longitude", "poi_name"]].isna().any(axis=1)
]  # poi_df[poi_df.isna().any(axis=1)]
con.print(
    f"WARNING: Gage {missing_meta_df.poi_gage_id.values} missing metadata and will be dropped from the poi_gdf."
)
# poi_df.notna(inplace=True, ignore_index=False)
# poi_df.reset_index(drop=True, inplace=True)

poi_gdf = gpd.GeoDataFrame(
    poi_df,
    geometry=gpd.points_from_xy(poi_df.longitude, poi_df.latitude),
    crs=crs,
).dropna(subset=["latitude", "longitude", "poi_name"])

In [ ]:
# poi_gdf.info()

In [ ]:
gages_list = gages_df.index.to_list()
nwis_gages_aoi_list = nwis_gages_aoi.poi_id.to_list()
nwis_gages_in_gages_list = list(set(nwis_gages_aoi_list) & set(gages_list))
con.print(
    f"The gages file has {len(gages_list)} streamflow gages, {len(nwis_gages_in_gages_list)} are in NWIS."
)

con.print(
    f"The parameter file has {len(poi_df)} gages, {len(poi_df) - len(poi_gdf)} are missing metadata and \ncannot be mapped in NHMassist notebooks unless the gages.csv is modified."
)

additional_gages = list(set(gages_list) - set(poi_df.poi_id))
nwis_gages_in_additional_gages_list = list(
    set(nwis_gages_aoi_list) & set(additional_gages)
)

con.print(
    f"An additional {len(additional_gages)} streamflow gages are in the model domain and not in the parameter file; {len(nwis_gages_in_additional_gages_list)} are in NWIS."
)
# nwis_gages_aoi_list = nwis_gages_aoi.poi_id.to_list()

In [ ]:
hru_gdb